In [ ]:
%pylab inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import combimuscle
import activation_signal
import muscle_utils
import numpy as np
import scipy as sc
from emg import EMGPOD

In [ ]:
import os
import pwd
user = pwd.getpwuid( os.getuid() )[0]
emg=np.genfromtxt(
    "/home/{}/Dropbox/exo-arm/logs/emg_logs/mr_morten/mr_morten_flex_isokinetic_3rep/emg-1485951230.csv".format(
        user
    ), 
    delimiter=',')

In [ ]:
emg4=emg[300:-300,4]
emg8=emg[300:-300,8]

emg4=sc.signal.resample(emg4,1658)
emg8=sc.signal.resample(emg8,1658)


a_lowerarm =[0]*1658 

In [ ]:
kinkom=np.genfromtxt(
    "/home/{}/Dropbox/exo-arm/logs/kincom_logs/morten/MRF1_clean".format(
        user
    )
    , delimiter=',')

In [ ]:
angle=abs(kinkom[100:-20,1])

In [ ]:
tau = []
a4_vec = []
activation_level_e4 = []
activation_level_e8 = []
for a, e4, e8 in zip(angle, emg4, emg8):
    angs = [a, 0]
    combimuscle.__add_meas__([0, 0, 0, e4, 0, 0, 0, e8])
    tau.append(combimuscle.get_comb_tau(angs, muscle_utils.MUSCLE_JOINT.ELBOW))
    
    a4, _, _, a8 = combimuscle.__get_activation_levels__()
    activation_level_e4.append(a4)
    activation_level_e8.append(a8)


In [ ]:
tau_kinkom = kinkom[100:-20,3] * 1/22
time = kinkom[100:-20,0]
plt.figure(1)
plt.plot(time, tau, 'x')
plt.plot(time, tau_kinkom, 'x')
plt.figure(2)
plt.plot(angle, tau, 'x')
plt.plot(angle, tau_kinkom, 'x')
plt.figure(3)
#plt.plot(activation_level_e4, tau, 'x')
plt.plot(activation_level_e4, tau_kinkom, 'x')

In [ ]:
from matplotlib import cm
x = activation_level_e4
y = angle
z = tau_kinkom

xi = np.linspace(min(x), max(x), 100)
yi = np.linspace(min(y), max(y), 100)

X, Y = np.meshgrid(xi, yi)
Z = griddata(x, y, z, xi, yi, interp='linear')


fig = plt.figure(4)
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)

In [ ]:
mean((tau - tau_kinkom)**2)